# Vertex AI Feature Store Migration Guide
## Legacy Feature Store (1.0) → Feature Store 2.0

---

### Overview

This notebook provides a complete migration path from Vertex AI Feature Store Legacy (1.0) to the new Feature Store 2.0 architecture. The migration uses BigQuery as an intermediate storage layer to transfer feature data between systems.

**Migration Flow:**  
`Legacy Feature Store` → `BigQuery Export` → `Feature Store 2.0`

---

### Quick Start

1. Configure project settings in **Configuration Settings** cell
2. Run all cells sequentially
3. Monitor sync completion and validate results

---

### Operations Summary

| Step | Operation | Description | Estimated Time |
|------|-----------|-------------|----------------|
| 1 | **Install Packages** | Install required Google Cloud libraries | 1-2 min |
| 2 | **Initialize Clients** | Create API clients for Feature Store, BigQuery, and Feature Store 2.0 | < 1 min |
| 3 | **Export to BigQuery** | Export all features from Legacy EntityType to BigQuery table | 2-10 min |
| 4 | **Validate Export** | Verify row count, schema, and data integrity | < 1 min |
| 5 | **Create Online Store** | Provision Feature Store 2.0 with Bigtable backend | 5-10 min |
| 6 | **Create FeatureView** | Link BigQuery table to Feature Store 2.0 | 1-2 min |
| 7 | **Trigger Sync** | Populate Feature Store 2.0 with exported data | 2-15 min |
| 8 | **Check Sync Status** | Review sync history and validate completion | < 1 min |

**Total Time:** 15-45 minutes (varies by data volume)

---

### Prerequisites

- ✓ GCP project with Vertex AI API enabled
- ✓ Existing Legacy Feature Store (1.0) with EntityType
- ✓ BigQuery dataset for intermediate storage
- ✓ IAM permissions: `Vertex AI Administrator`, `BigQuery User`
- ✓ Sufficient quota for Bigtable nodes in target region

---

### Important Notes

- This is a **reference implementation** for migration purposes
- The migration is **non-destructive** (legacy store remains unchanged)
- Feature Store 2.0 supports scheduled syncs via cron configuration
- Embedding management can be enabled for vector similarity search

---

### References

- [Feature Store Migration Documentation](https://cloud.google.com/vertex-ai/docs/featurestore#migrate)
- [Vertex AI Samples Repository](https://github.com/GoogleCloudPlatform/vertex-ai-samples)
- [Feature Store 2.0 Overview](https://cloud.google.com/vertex-ai/docs/featurestore/latest/overview)

---

**Questions?:** mateuswagner@google.com  
**Last Updated:** 2025-11  
**Status:** Not an Official Google Product

## Part 1: Export Legacy Feature Store to BigQuery

### Step 1: Install Required Packages

In [ ]:
%pip install --upgrade google-cloud-aiplatform google-cloud-bigquery

In [ ]:
import logging
import time
from typing import Optional

from google.cloud import bigquery
from google.cloud.aiplatform_v1beta1 import (
    FeaturestoreServiceClient,
    FeatureValueDestination,
    FeatureSelector,
    ExportFeatureValuesRequest,
    ListFeatureViewSyncsRequest,
    types,
)
from google.cloud.aiplatform_v1beta1 import (
    FeatureOnlineStoreAdminServiceClient,
    FeatureOnlineStoreServiceClient,
)
from google.cloud.aiplatform_v1beta1.types import (
    feature_online_store as feature_online_store_pb2,
    feature_online_store_admin_service as feature_online_store_admin_service_pb2,
    feature_online_store_service as feature_online_store_service_pb2,
    feature_view as feature_view_pb2,
)
from google.api_core import exceptions

import numpy as np
import pandas as pd
from google.cloud.aiplatform_v1beta1 import (
    FeaturestoreServiceClient,
    Featurestore,
    EntityType,
    Feature,
)
from google.cloud.aiplatform_v1beta1.types import (
    featurestore_service as featurestore_service_pb2,
    entity_type as entity_type_pb2,
    feature as feature_pb2,
)

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

In [2]:
# Configuration Settings
# Update these values according to your GCP environment

# GCP Project Configuration
PROJECT_ID = "matt-demos"
REGION_ID = "us-central1"

# Legacy Feature Store Configuration (Source)
FEATURE_STORE_ID = "legacy_fs_1"
ENTITYTYPE_ID = "benchmark_3emb_1000xfloat64"

# BigQuery Configuration (Intermediate Storage)
BIGQUERY_DEST_DATASET_ID = "featurestore_ds"

# Feature Store 2.0 Configuration (Destination)
FEATURE_ONLINE_STORE_ID = "featurestore_4"
FEATURE_ONLINE_STORE_EMBEDDING_MNGMT = True

# Bigtable Auto-scaling Configuration
BIGTABLE_MIN_NODE_COUNT = 1
BIGTABLE_MAX_NODE_COUNT = 2
BIGTABLE_CPU_UTILIZATION_TARGET = 50

# Sync Schedule (Cron format with timezone)
# Example: Run at 56 minutes past every hour, Pacific Time
CRON_SCHEDULE = "TZ=America/Los_Angeles 56 * * * *"

# Operation Timeouts
DEFAULT_TIMEOUT = 3600  # 1 hour in seconds
SYNC_POLL_INTERVAL = 30  # 30 seconds

# Logging Configuration
LOG_LEVEL = logging.INFO  # Use logging.DEBUG for troubleshooting

logger.setLevel(LOG_LEVEL)

In [4]:
# Derived Variables - Do not modify unless you know what you're doing

# Parent resource path for the region
parent_id = f'projects/{PROJECT_ID}/locations/{REGION_ID}'

# Full path to the legacy EntityType
entitytype_id_path = f'{parent_id}/featurestores/{FEATURE_STORE_ID}/entityTypes/{ENTITYTYPE_ID}'

# Extract short entity type name
entity_type_short_id = entitytype_id_path.split("/")[-1]

# BigQuery destination URI
bigquery_output_url = f'bq://{PROJECT_ID}.{BIGQUERY_DEST_DATASET_ID}.{entity_type_short_id}'

# Entity ID column name in BigQuery (automatically generated by export)
entity_id_column = f'entity_type_{entity_type_short_id}'

# Feature Online Store 2.0 resource paths
feature_online_stores_name = f"{parent_id}/featureOnlineStores/{FEATURE_ONLINE_STORE_ID}"
feature_view_id = entity_type_short_id
feature_view_full = f"{feature_online_stores_name}/featureViews/{feature_view_id}"

logger.info("Configuration initialized successfully")
logger.info(f"Source: {entitytype_id_path}")
logger.info(f"Destination: {feature_online_stores_name}/{feature_view_id}")

2025-11-20 17:18:13,446 - INFO - Configuration initialized successfully
2025-11-20 17:18:13,447 - INFO - Source: projects/matt-demos/locations/us-central1/featurestores/legacy_fs_1/entityTypes/benchmark_3emb_1000xfloat64
2025-11-20 17:18:13,447 - INFO - Destination: projects/matt-demos/locations/us-central1/featureOnlineStores/featurestore_4/benchmark_3emb_1000xfloat64
2025-11-20 17:18:13,447 - INFO - Source: projects/matt-demos/locations/us-central1/featurestores/legacy_fs_1/entityTypes/benchmark_3emb_1000xfloat64
2025-11-20 17:18:13,447 - INFO - Destination: projects/matt-demos/locations/us-central1/featureOnlineStores/featurestore_4/benchmark_3emb_1000xfloat64


### Step 0: Create Demo Source Feature Store (Optional)

**Run this cell only if you need to create a demo Legacy Feature Store with fake data for testing purposes.**

In [ ]:
import numpy as np
import pandas as pd
from google.cloud.aiplatform_v1beta1 import (
    FeaturestoreServiceClient,
    Featurestore,
    EntityType,
    Feature,
)
from google.cloud.aiplatform_v1beta1.types import (
    featurestore_service as featurestore_service_pb2,
    entity_type as entity_type_pb2,
    feature as feature_pb2,
    featurestore_online_service as featurestore_online_service_pb2,
)
from google.protobuf.struct_pb2 import Value, ListValue

try:
    logger.info("=" * 80)
    logger.info("CREATING DEMO LEGACY FEATURE STORE")
    logger.info("=" * 80)
    
    # Initialize Legacy Feature Store client
    demo_fs_client = FeaturestoreServiceClient(
        client_options={"api_endpoint": f'{REGION_ID}-aiplatform.googleapis.com'}
    )
    
    # Derive parent_id (in case not yet set)
    parent_id = f'projects/{PROJECT_ID}/locations/{REGION_ID}'
    
    # Step 1: Create Legacy Feature Store
    featurestore_path = f"{parent_id}/featurestores/{FEATURE_STORE_ID}"
    
    try:
        existing_fs = demo_fs_client.get_featurestore(name=featurestore_path)
        logger.info(f"✓ Feature Store '{FEATURE_STORE_ID}' already exists")
    except exceptions.NotFound:
        logger.info(f"Creating Legacy Feature Store: {FEATURE_STORE_ID}")
        create_fs_op = demo_fs_client.create_featurestore(
            featurestore_service_pb2.CreateFeaturestoreRequest(
                parent=parent_id,
                featurestore_id=FEATURE_STORE_ID,
                featurestore=Featurestore(
                    online_serving_config=Featurestore.OnlineServingConfig(
                        fixed_node_count=1
                    )
                )
            )
        )
        create_fs_op.result(timeout=600)
        logger.info(f"✓ Feature Store '{FEATURE_STORE_ID}' created")
    
    # Step 2: Create EntityType
    entitytype_id_path = f'{parent_id}/featurestores/{FEATURE_STORE_ID}/entityTypes/{ENTITYTYPE_ID}'
    
    try:
        existing_et = demo_fs_client.get_entity_type(name=entitytype_id_path)
        logger.info(f"✓ EntityType '{ENTITYTYPE_ID}' already exists")
    except exceptions.NotFound:
        logger.info(f"Creating EntityType: {ENTITYTYPE_ID}")
        create_et_op = demo_fs_client.create_entity_type(
            featurestore_service_pb2.CreateEntityTypeRequest(
                parent=featurestore_path,
                entity_type_id=ENTITYTYPE_ID,
                entity_type=EntityType()
            )
        )
        create_et_op.result(timeout=300)
        logger.info(f"✓ EntityType '{ENTITYTYPE_ID}' created")
    
    # Step 3: Create Features (3 embeddings with 1000 float64 dimensions each)
    feature_specs = [
        ("embedding_1", "DOUBLE", 1000),
        ("embedding_2", "DOUBLE", 1000),
        ("embedding_3", "DOUBLE", 1000),
    ]
    
    for feature_name, value_type, array_size in feature_specs:
        feature_path = f"{entitytype_id_path}/features/{feature_name}"
        
        try:
            demo_fs_client.get_feature(name=feature_path)
            logger.info(f"✓ Feature '{feature_name}' already exists")
        except exceptions.NotFound:
            logger.info(f"Creating feature: {feature_name} (ARRAY<{value_type}>[{array_size}])")
            
            # Create feature with array type
            create_feature_op = demo_fs_client.create_feature(
                featurestore_service_pb2.CreateFeatureRequest(
                    parent=entitytype_id_path,
                    feature_id=feature_name,
                    feature=Feature(
                        value_type=Feature.ValueType.DOUBLE_ARRAY,
                    )
                )
            )
            create_feature_op.result(timeout=300)
            logger.info(f"✓ Feature '{feature_name}' created")
    
    # Step 4: Generate and ingest fake embedding data
    logger.info("")
    logger.info("Generating fake embedding data...")
    
    NUM_ENTITIES = 100  # Generate 100 sample entities
    EMBEDDING_DIM = 1000
    
    # Generate random entity IDs
    entity_ids = [f"entity_{i:04d}" for i in range(NUM_ENTITIES)]
    
    # Generate random embeddings (normalized to unit vectors for realism)
    embeddings_data = []
    for entity_id in entity_ids:
        # Create 3 random embeddings
        emb1 = np.random.randn(EMBEDDING_DIM).astype(np.float64)
        emb2 = np.random.randn(EMBEDDING_DIM).astype(np.float64)
        emb3 = np.random.randn(EMBEDDING_DIM).astype(np.float64)
        
        # Normalize to unit vectors
        emb1 = emb1 / np.linalg.norm(emb1)
        emb2 = emb2 / np.linalg.norm(emb2)
        emb3 = emb3 / np.linalg.norm(emb3)
        
        embeddings_data.append({
            "entity_id": entity_id,
            "embedding_1": emb1.tolist(),
            "embedding_2": emb2.tolist(),
            "embedding_3": emb3.tolist(),
        })
    
    logger.info(f"Generated {NUM_ENTITIES} entities with 3 embeddings each ({EMBEDDING_DIM} dimensions)")
    
    # Ingest data in batches using streaming write API
    logger.info("Ingesting data into Legacy Feature Store...")
    logger.info("Note: Using streaming ingestion for demo purposes")
    
    # Batch ingestion (10 entities at a time)
    BATCH_SIZE = 10
    total_batches = (NUM_ENTITIES + BATCH_SIZE - 1) // BATCH_SIZE
    
    for batch_idx in range(0, NUM_ENTITIES, BATCH_SIZE):
        batch_entities = embeddings_data[batch_idx:batch_idx + BATCH_SIZE]
        
        # Prepare payloads for this batch
        payloads = []
        for entity_data in batch_entities:
            # Create feature values dictionary using protobuf Value
            feature_values = {}
            
            # Convert embeddings to ListValue
            for emb_name in ["embedding_1", "embedding_2", "embedding_3"]:
                list_value = ListValue()
                list_value.values.extend([Value(number_value=v) for v in entity_data[emb_name]])
                feature_values[emb_name] = featurestore_online_service_pb2.FeatureValue(
                    value=Value(list_value=list_value)
                )
            
            payloads.append(
                featurestore_service_pb2.WriteFeatureValuesPayload(
                    entity_id=entity_data["entity_id"],
                    feature_values=feature_values
                )
            )
        
        # Write this batch
        demo_fs_client.write_feature_values(
            featurestore_service_pb2.WriteFeatureValuesRequest(
                entity_type=entitytype_id_path,
                payloads=payloads
            )
        )
        
        current_batch = (batch_idx // BATCH_SIZE) + 1
        logger.info(f"  Ingested batch {current_batch}/{total_batches} ({len(batch_entities)} entities)")
    
    logger.info("")
    logger.info("=" * 80)
    logger.info("✓ DEMO LEGACY FEATURE STORE SETUP COMPLETE")
    logger.info("=" * 80)
    logger.info(f"Feature Store: {FEATURE_STORE_ID}")
    logger.info(f"EntityType: {ENTITYTYPE_ID}")
    logger.info(f"Features: 3 embeddings × {EMBEDDING_DIM} dimensions")
    logger.info(f"Entities ingested: {NUM_ENTITIES}")
    logger.info("")
    logger.info("You can now proceed with the export process below.")
    logger.info("=" * 80)
    
except Exception as e:
    logger.error(f"Failed to create demo feature store: {e}")
    logger.error("You may need to manually create the Legacy Feature Store")
    raise

### Step 2: Initialize API Clients

In [ ]:
try:
    # Initialize Legacy Feature Store client (v1beta1)
    fs_client = FeaturestoreServiceClient(
        client_options={
            "api_endpoint": f'{REGION_ID}-aiplatform.googleapis.com'
        }
    )
    logger.info("✓ Legacy Feature Store client initialized")

    # Initialize BigQuery client
    bq_client = bigquery.Client(project=PROJECT_ID)
    logger.info("✓ BigQuery client initialized")

    # Initialize Feature Store 2.0 Admin client
    admin_vfs2_client = FeatureOnlineStoreAdminServiceClient(
        client_options={
            "api_endpoint": f'{REGION_ID}-aiplatform.googleapis.com'
        }
    )
    logger.info("✓ Feature Store 2.0 Admin client initialized")
    
    logger.info("All clients initialized successfully")
    
except Exception as e:
    logger.error(f"Failed to initialize clients: {e}")
    raise

### Step 3: Export EntityType from Legacy Feature Store to BigQuery

This step exports all feature values from the legacy EntityType to a BigQuery table.

In [ ]:
try:
    # Configure BigQuery destination
    destination = FeatureValueDestination()
    destination.bigquery_destination.output_uri = bigquery_output_url
    
    # Select all features using wildcard matcher
    feature_selector = FeatureSelector()
    feature_selector.id_matcher.ids = ['*']
    
    logger.info(f"Starting export from EntityType: {ENTITYTYPE_ID}")
    logger.info(f"Destination: {bigquery_output_url}")
    logger.info("This may take several minutes depending on data size...")
    
    # Start export operation
    operation = fs_client.export_feature_values(
        ExportFeatureValuesRequest(
            entity_type=entitytype_id_path,
            destination=destination,
            feature_selector=feature_selector,
            full_export=types.ExportFeatureValuesRequest.FullExport()
        ),
        timeout=DEFAULT_TIMEOUT
    )
    
    # Wait for export to complete
    result = operation.result()
    
    logger.info("✓ Export completed successfully")
    logger.info(f"Entity type '{entity_type_short_id}' from feature store '{FEATURE_STORE_ID}'")
    logger.info(f"Exported to BigQuery: {bigquery_output_url}")
    
except exceptions.NotFound as e:
    logger.error(f"Resource not found: {e}")
    logger.error("Please verify FEATURE_STORE_ID and ENTITYTYPE_ID are correct")
    raise
except exceptions.PermissionDenied as e:
    logger.error(f"Permission denied: {e}")
    logger.error("Ensure you have the necessary IAM permissions")
    raise
except Exception as e:
    logger.error(f"Export failed: {e}")
    raise

### Step 4: Validate Exported Data

Verify the export by checking row count and schema of the BigQuery table.

In [ ]:
try:
    table_ref = f'{PROJECT_ID}.{BIGQUERY_DEST_DATASET_ID}.{ENTITYTYPE_ID}'
    table = bq_client.get_table(table_ref)
    
    logger.info("=" * 80)
    logger.info("EXPORT VALIDATION")
    logger.info("=" * 80)
    logger.info(f"Table: {table_ref}")
    logger.info(f"Total Rows: {table.num_rows:,}")
    logger.info(f"Total Columns: {len(table.schema)}")
    logger.info(f"Table Size: {table.num_bytes / (1024**3):.2f} GB")
    logger.info(f"Created: {table.created}")
    logger.info(f"Modified: {table.modified}")
    logger.info("")
    logger.info("Schema (first 10 columns):")
    for i, field in enumerate(table.schema[:10]):
        logger.info(f"  {i+1}. {field.name} ({field.field_type})")
    
    if len(table.schema) > 10:
        logger.info(f"  ... and {len(table.schema) - 10} more columns")
    
    logger.info("=" * 80)
    
    # Validate entity ID column exists
    column_names = [field.name for field in table.schema]
    if entity_id_column not in column_names:
        logger.warning(f"Expected entity ID column '{entity_id_column}' not found!")
        logger.info(f"Available columns: {', '.join(column_names[:5])}...")
    else:
        logger.info(f"✓ Entity ID column '{entity_id_column}' found")
        
except Exception as e:
    logger.error(f"Failed to validate exported table: {e}")
    raise

## Part 2: Create Feature Store 2.0 and Import Data

### Step 5: Create Feature Online Store 2.0

This creates a new Feature Online Store backed by Bigtable with auto-scaling enabled.

In [ ]:
try:
    # Check if online store already exists
    try:
        existing_store = admin_vfs2_client.get_feature_online_store(
            name=feature_online_stores_name
        )
        logger.info(f"✓ Feature Online Store '{FEATURE_ONLINE_STORE_ID}' already exists")
        logger.info(f"State: {existing_store.state.name}")
        logger.info("Skipping creation step")
        
    except exceptions.NotFound:
        logger.info(f"Creating Feature Online Store: {FEATURE_ONLINE_STORE_ID}")
        logger.info("This may take 5-10 minutes...")
        
        # Configure Feature Online Store with Bigtable backend
        online_store_config = feature_online_store_pb2.FeatureOnlineStore(
            bigtable=feature_online_store_pb2.FeatureOnlineStore.Bigtable(
                auto_scaling=feature_online_store_pb2.FeatureOnlineStore.Bigtable.AutoScaling(
                    min_node_count=BIGTABLE_MIN_NODE_COUNT,
                    max_node_count=BIGTABLE_MAX_NODE_COUNT,
                    cpu_utilization_target=BIGTABLE_CPU_UTILIZATION_TARGET
                )
            ),
            embedding_management=feature_online_store_pb2.FeatureOnlineStore.EmbeddingManagement(
                enabled=FEATURE_ONLINE_STORE_EMBEDDING_MNGMT
            )
        )
        
        # Create the online store
        create_store_lro = admin_vfs2_client.create_feature_online_store(
            feature_online_store_admin_service_pb2.CreateFeatureOnlineStoreRequest(
                parent=parent_id,
                feature_online_store_id=FEATURE_ONLINE_STORE_ID,
                feature_online_store=online_store_config,
            )
        )
        
        # Wait for creation to complete
        result = create_store_lro.result()
        logger.info("✓ Feature Online Store created successfully")
        logger.info(f"Name: {result.name}")
        logger.info(f"State: {result.state.name}")
        
except exceptions.AlreadyExists:
    logger.info(f"✓ Feature Online Store '{FEATURE_ONLINE_STORE_ID}' already exists")
except Exception as e:
    logger.error(f"Failed to create Feature Online Store: {e}")
    raise

### Step 6: Create FeatureView

A FeatureView links the BigQuery table to the Feature Online Store and defines the sync schedule.

In [ ]:
try:
    # Check if FeatureView already exists
    try:
        existing_view = admin_vfs2_client.get_feature_view(name=feature_view_full)
        logger.info(f"✓ FeatureView '{feature_view_id}' already exists")
        logger.info(f"State: {existing_view.state.name}")
        logger.info("Skipping creation step")
        
    except exceptions.NotFound:
        logger.info(f"Creating FeatureView: {feature_view_id}")
        logger.info(f"Source: {bigquery_output_url}")
        logger.info(f"Entity ID column: {entity_id_column}")
        logger.info(f"Sync schedule: {CRON_SCHEDULE}")
        
        # Configure BigQuery source
        big_query_source = feature_view_pb2.FeatureView.BigQuerySource(
            uri=bigquery_output_url,
            entity_id_columns=[entity_id_column]
        )
        
        # Configure sync schedule
        sync_config = feature_view_pb2.FeatureView.SyncConfig(cron=CRON_SCHEDULE)
        
        # Create FeatureView
        create_view_lro = admin_vfs2_client.create_feature_view(
            feature_online_store_admin_service_pb2.CreateFeatureViewRequest(
                parent=feature_online_stores_name,
                feature_view_id=feature_view_id,
                feature_view=feature_view_pb2.FeatureView(
                    big_query_source=big_query_source,
                    sync_config=sync_config,
                )
            )
        )
        
        # Wait for creation to complete
        result = create_view_lro.result()
        logger.info("✓ FeatureView created successfully")
        logger.info(f"Name: {result.name}")
        logger.info(f"State: {result.state.name}")
        
except exceptions.AlreadyExists:
    logger.info(f"✓ FeatureView '{feature_view_id}' already exists")
except exceptions.InvalidArgument as e:
    logger.error(f"Invalid argument: {e}")
    logger.error("Check that BigQuery table exists and entity_id_column is correct")
    raise
except Exception as e:
    logger.error(f"Failed to create FeatureView: {e}")
    raise

### Step 7: Trigger On-Demand Sync

Start an immediate sync to populate the Feature Online Store with data from BigQuery.

In [ ]:
try:
    logger.info(f"Starting on-demand sync for FeatureView: {feature_view_id}")
    logger.info("This may take several minutes depending on data size...")
    
    # Trigger sync
    sync_response = admin_vfs2_client.sync_feature_view(
        feature_view=feature_view_full
    )
    
    logger.info(f"Sync job started: {sync_response.feature_view_sync}")
    
    # Poll for sync completion
    poll_count = 0
    start_time = time.time()
    
    while True:
        feature_view_sync = admin_vfs2_client.get_feature_view_sync(
            name=sync_response.feature_view_sync
        )
        
        # Check if sync has completed
        if feature_view_sync.run_time.end_time.seconds > 0:
            elapsed_time = time.time() - start_time
            status = "Succeeded" if feature_view_sync.final_status.code == 0 else "Failed"
            
            logger.info("=" * 80)
            logger.info(f"SYNC {status.upper()}")
            logger.info("=" * 80)
            logger.info(f"Sync Name: {feature_view_sync.name}")
            logger.info(f"Status Code: {feature_view_sync.final_status.code}")
            logger.info(f"Start Time: {feature_view_sync.run_time.start_time}")
            logger.info(f"End Time: {feature_view_sync.run_time.end_time}")
            logger.info(f"Duration: {elapsed_time:.1f} seconds")
            
            if feature_view_sync.final_status.message:
                logger.info(f"Message: {feature_view_sync.final_status.message}")
            
            logger.info("=" * 80)
            
            if feature_view_sync.final_status.code != 0:
                raise Exception(f"Sync failed: {feature_view_sync.final_status.message}")
            
            # Wait a bit for the system to stabilize
            logger.info("Waiting 30 seconds for system stabilization...")
            time.sleep(30)
            break
        else:
            poll_count += 1
            elapsed = time.time() - start_time
            logger.info(f"Sync in progress... ({elapsed:.0f}s elapsed, poll #{poll_count})")
            time.sleep(SYNC_POLL_INTERVAL)
            
except Exception as e:
    logger.error(f"Sync operation failed: {e}")
    raise

### Step 8: Check Sync History

View all sync operations for this FeatureView.

In [ ]:
try:
    logger.info("Fetching sync history...")
    
    request = ListFeatureViewSyncsRequest(parent=feature_view_full)
    page_result = admin_vfs2_client.list_feature_view_syncs(request=request)
    
    syncs = list(page_result)
    
    logger.info("=" * 80)
    logger.info(f"SYNC HISTORY ({len(syncs)} total syncs)")
    logger.info("=" * 80)
    
    for i, sync in enumerate(syncs, 1):
        status = "Success" if sync.final_status.code == 0 else "Failed"
        duration = "N/A"
        
        if sync.run_time.end_time.seconds > 0:
            duration_seconds = (
                sync.run_time.end_time.seconds - sync.run_time.start_time.seconds
            )
            duration = f"{duration_seconds}s"
        
        logger.info(f"\n{i}. {sync.name.split('/')[-1]}")
        logger.info(f"   Status: {status} (code: {sync.final_status.code})")
        logger.info(f"   Start: {sync.run_time.start_time}")
        logger.info(f"   Duration: {duration}")
        
        if sync.final_status.message:
            logger.info(f"   Message: {sync.final_status.message}")
    
    logger.info("=" * 80)
    
except Exception as e:
    logger.error(f"Failed to fetch sync history: {e}")
    raise

## Cleanup (Optional)

### Delete Feature Online Store

**WARNING**: This will permanently delete the Feature Online Store and all associated data.  
Uncomment and run the cell below only if you want to remove the resources.

In [ ]:
# Uncomment to delete the Feature Online Store
# try:
#     logger.info(f"Deleting Feature Online Store: {FEATURE_ONLINE_STORE_ID}")
#     logger.info("This may take several minutes...")
#     
#     delete_op = admin_vfs2_client.delete_feature_online_store(
#         name=f"projects/{PROJECT_ID}/locations/{REGION_ID}/featureOnlineStores/{FEATURE_ONLINE_STORE_ID}",
#         force=True,
#     )
#     
#     delete_op.result()
#     logger.info("✓ Feature Online Store deleted successfully")
#     
# except exceptions.NotFound:
#     logger.warning("Feature Online Store not found (may already be deleted)")
# except Exception as e:
#     logger.error(f"Failed to delete Feature Online Store: {e}")
#     raise